[**ResNeXt Blocks (Xie et al., 2017):**](https://openaccess.thecvf.com/content_cvpr_2017/papers/Xie_Aggregated_Residual_Transformations_CVPR_2017_paper.pdf) is an advancement for sparser connections, extending ResNet with grouped convolutions to enhance efficiency and performance.

<div>
<img src="./imgs/resnext.png" style="width: 800px;">
</div>

ResNeXt is an extension of ResNet that introduces grouped convolutions through a hyperparameter called cardinality, allowing the model to learn more diverse features efficiently.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import utils

In [3]:
class ResNeXtBlock(nn.Module):
    def __init__(self, in_channels, bottleneck_channels, out_channels, stride=1, cardinality=32):
        """ ResNeXt Block with grouped convolutions. """
        super().__init__()

        # 1x1 convolution to reduce channels
        self.conv1 = nn.Conv2d(in_channels, bottleneck_channels, kernel_size=1, stride=1)
        self.bn1 = nn.BatchNorm2d(bottleneck_channels)
        self.relu = nn.ReLU(inplace=True)

        # 3x3 grouped convolution with cardinality
        self.conv2 = nn.Conv2d(
            bottleneck_channels, bottleneck_channels, kernel_size=3, stride=stride, padding=1, groups=cardinality
        )
        self.bn2 = nn.BatchNorm2d(bottleneck_channels)

        # 1x1 convolution to restore channels
        self.conv3 = nn.Conv2d(bottleneck_channels, out_channels, kernel_size=1, stride=1)
        self.bn3 = nn.BatchNorm2d(out_channels)
        
        # Shortcut connection
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.shortcut = nn.Identity()

    def forward(self, x):
        # Main path
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))

        # Add shortcut
        out += self.shortcut(x)
        out = self.relu(out)
        
        return out

In [4]:
class ResNeXt(nn.Module):
    def __init__(self, block, layers, cardinality=32, num_classes=1000):
        super().__init__()
        self.cardinality = cardinality

        # Channel configurations for the stages
        self.channels = [256, 512, 1024, 2048]

        # Stem
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Four stages
        self.layer1 = self.make_stage(block, 64, self.channels[0], layers[0], stride=1)
        self.layer2 = self.make_stage(block, self.channels[0], self.channels[1], layers[1], stride=2)
        self.layer3 = self.make_stage(block, self.channels[1], self.channels[2], layers[2], stride=2)
        self.layer4 = self.make_stage(block, self.channels[2], self.channels[3], layers[3], stride=2)

        # Global average pooling and fully connected layer
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(self.channels[3], num_classes)

    def make_stage(self, block, in_channels, out_channels, num_blocks, stride):
        """Helper function to create a stage of ResNeXt blocks."""
        layers = []
        bottleneck_channels = out_channels // 4  # Bottleneck ratio of 4
        # First block with specified stride
        layers.append(block(in_channels, bottleneck_channels, out_channels, stride, self.cardinality))
        # Subsequent blocks with stride 1
        for _ in range(1, num_blocks):
            layers.append(block(out_channels, bottleneck_channels, out_channels, 1, self.cardinality))
        return nn.Sequential(*layers)

    def forward(self, x):
        # Stem
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        # Stages
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        # Pooling and classification
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [6]:
def ResNeXt50(cardinality=32, num_classes=10):
    """ResNeXt-50: 4 stages with 3, 4, 6, 3 blocks."""
    return ResNeXt(ResNeXtBlock, [3, 4, 6, 3], cardinality, num_classes)

def ResNeXt101(cardinality=32, num_classes=10):
    """ResNeXt-101: 4 stages with 3, 4, 23, 3 blocks."""
    return ResNeXt(ResNeXtBlock, [3, 4, 23, 3], cardinality, num_classes)

In [7]:
utils.layer_summary(ResNeXt50(num_classes=10), (1, 3, 224, 224))

Layer Name                     Layer Type              Param #         Output Shape
conv1                          Conv2d                     9408    (1, 64, 112, 112)
bn1                            BatchNorm2d                 128    (1, 64, 112, 112)
relu                           ReLU                          0    (1, 64, 112, 112)
maxpool                        MaxPool2d                     0      (1, 64, 56, 56)
layer1.0.conv1                 Conv2d                     4160      (1, 64, 56, 56)
layer1.0.bn1                   BatchNorm2d                 128      (1, 64, 56, 56)
layer1.0.relu                  ReLU                          0      (1, 64, 56, 56)
layer1.0.conv2                 Conv2d                     1216      (1, 64, 56, 56)
layer1.0.bn2                   BatchNorm2d                 128      (1, 64, 56, 56)
layer1.0.conv3                 Conv2d                    16640     (1, 256, 56, 56)
layer1.0.bn3                   BatchNorm2d                 512     (1, 256, 

In [8]:
utils.layer_summary(ResNeXt101(num_classes=10), (1, 3, 224, 224))

Layer Name                     Layer Type              Param #         Output Shape
conv1                          Conv2d                     9408    (1, 64, 112, 112)
bn1                            BatchNorm2d                 128    (1, 64, 112, 112)
relu                           ReLU                          0    (1, 64, 112, 112)
maxpool                        MaxPool2d                     0      (1, 64, 56, 56)
layer1.0.conv1                 Conv2d                     4160      (1, 64, 56, 56)
layer1.0.bn1                   BatchNorm2d                 128      (1, 64, 56, 56)
layer1.0.relu                  ReLU                          0      (1, 64, 56, 56)
layer1.0.conv2                 Conv2d                     1216      (1, 64, 56, 56)
layer1.0.bn2                   BatchNorm2d                 128      (1, 64, 56, 56)
layer1.0.conv3                 Conv2d                    16640     (1, 256, 56, 56)
layer1.0.bn3                   BatchNorm2d                 512     (1, 256, 

In [9]:
data = utils.CIFAR10DataLoader(batch_size=64, resize=(224, 224))
train_loader = data.get_train_loader()
test_loader = data.get_test_loader()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNeXt50(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
for epoch in range(epochs):
    train_loss, train_acc = utils.train_step(train_loader, model, criterion, optimizer, device)
    test_loss, test_acc = utils.eval_step(test_loader, model, criterion, device)
    print(f"Epoch {epoch + 1}/{epochs}: Train Loss={train_loss}, Test Loss={test_loss}, Test Accuracy={test_acc}")